The dataset is obtained from TalkingData AdTracking competition. The goal of the competition is to predict if a user will download an app after clicking through an ad.

In [1]:
import pandas as pd
import numpy as np

click_data = pd.read_csv('train_TalkingData_AdTracking.csv', parse_dates=['click_time'])
click_data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,89489,3,1,13,379,2017-11-06 15:13:23,NaN,0
1,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1
2,3437,6,1,13,459,2017-11-06 15:42:32,NaN,0
3,167543,3,1,13,379,2017-11-06 15:56:17,NaN,0
4,147509,3,1,13,379,2017-11-06 15:57:01,NaN,0


In [2]:
# Add new columns for timestamp features day, hour, minute, and second
clicks = click_data.copy()
clicks['day'] = clicks['click_time'].dt.day.astype('uint8')
clicks

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day
0,89489,3,1,13,379,2017-11-06 15:13:23,NaN,0,6
1,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6
2,3437,6,1,13,459,2017-11-06 15:42:32,NaN,0,6
3,167543,3,1,13,379,2017-11-06 15:56:17,NaN,0,6
4,147509,3,1,13,379,2017-11-06 15:57:01,NaN,0,6
...,...,...,...,...,...,...,...,...,...
2300556,32457,2,1,19,477,2017-11-09 15:59:59,NaN,0,9
2300557,20266,14,1,13,446,2017-11-09 15:59:59,NaN,0,9
2300558,49383,12,2,17,178,2017-11-09 16:00:00,NaN,0,9
2300559,34894,12,1,15,145,2017-11-09 16:00:00,NaN,0,9


In [5]:
clicks['year'] = clicks['click_time'].dt.year.astype('uint8')
clicks['month'] = clicks['click_time'].dt.month.astype('uint8')
clicks['hour'] = clicks['click_time'].dt.hour.astype('uint8')
clicks['minute'] = clicks['click_time'].dt.minute.astype('uint8')
clicks['second'] = clicks['click_time'].dt.second.astype('uint8')
clicks

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,year,month,hour,minute,second
0,89489,3,1,13,379,2017-11-06 15:13:23,NaN,0,6,225,11,15,13,23
1,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,225,11,15,41,7
2,3437,6,1,13,459,2017-11-06 15:42:32,NaN,0,6,225,11,15,42,32
3,167543,3,1,13,379,2017-11-06 15:56:17,NaN,0,6,225,11,15,56,17
4,147509,3,1,13,379,2017-11-06 15:57:01,NaN,0,6,225,11,15,57,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300556,32457,2,1,19,477,2017-11-09 15:59:59,NaN,0,9,225,11,15,59,59
2300557,20266,14,1,13,446,2017-11-09 15:59:59,NaN,0,9,225,11,15,59,59
2300558,49383,12,2,17,178,2017-11-09 16:00:00,NaN,0,9,225,11,16,0,0
2300559,34894,12,1,15,145,2017-11-09 16:00:00,NaN,0,9,225,11,16,0,0


In [6]:
from sklearn import preprocessing

cat_features = ['ip', 'app', 'device', 'os', 'channel']

label_encoder = preprocessing.LabelEncoder()

# Create new columns in clicks using preprocessing.LabelEncoder()
for feature in cat_features:
    encoded_feature = label_encoder.fit_transform(clicks[feature])
    clicks[feature + '_labels'] = encoded_feature

Now it suppose to have additional columns. For each of the categorical features ['ip', 'app', 'device', 'os', 'channel'], the new columns are created based on original column name with add-on '_labels' appended. For example, **`ip_labels`**.

In [7]:
clicks

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,day,year,month,hour,minute,second,ip_labels,app_labels,device_labels,os_labels,channel_labels
0,89489,3,1,13,379,2017-11-06 15:13:23,NaN,0,6,225,11,15,13,23,27226,3,1,13,120
1,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1,6,225,11,15,41,7,110007,35,1,13,10
2,3437,6,1,13,459,2017-11-06 15:42:32,NaN,0,6,225,11,15,42,32,1047,6,1,13,157
3,167543,3,1,13,379,2017-11-06 15:56:17,NaN,0,6,225,11,15,56,17,76270,3,1,13,120
4,147509,3,1,13,379,2017-11-06 15:57:01,NaN,0,6,225,11,15,57,1,57862,3,1,13,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300556,32457,2,1,19,477,2017-11-09 15:59:59,NaN,0,9,225,11,15,59,59,9791,2,1,19,166
2300557,20266,14,1,13,446,2017-11-09 15:59:59,NaN,0,9,225,11,15,59,59,6240,14,1,13,146
2300558,49383,12,2,17,178,2017-11-09 16:00:00,NaN,0,9,225,11,16,0,0,15098,12,2,17,50
2300559,34894,12,1,15,145,2017-11-09 16:00:00,NaN,0,9,225,11,16,0,0,10538,12,1,15,41


To create training, validation, and test splits. First, clicks DataFrame is sorted in order of increasing time. The first 80% of the rows are the train set, the next 10% are the validation set, and the last 10% are the test set.

In [8]:
feature_cols = ['day', 'hour', 'minute', 'second', 
                'ip_labels', 'app_labels', 'device_labels',
                'os_labels', 'channel_labels']

valid_fraction = 0.1
clicks_srt = clicks.sort_values('click_time')
valid_rows = int(len(clicks_srt) * valid_fraction)
train = clicks_srt[:-valid_rows * 2]

# valid size == test size, last two sections of the data
valid = clicks_srt[-valid_rows * 2:-valid_rows]
test = clicks_srt[-valid_rows:]

In [10]:
import lightgbm as lgb

dtrain = lgb.Dataset(train[feature_cols], label=train['is_attributed'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['is_attributed'])
dtest = lgb.Dataset(test[feature_cols], label=test['is_attributed'])

param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 100

bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10)

[1]	valid_0's auc: 0.948979
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.949235
[3]	valid_0's auc: 0.950126
[4]	valid_0's auc: 0.950072
[5]	valid_0's auc: 0.950536
[6]	valid_0's auc: 0.950943
[7]	valid_0's auc: 0.951453
[8]	valid_0's auc: 0.951518
[9]	valid_0's auc: 0.952385
[10]	valid_0's auc: 0.952434
[11]	valid_0's auc: 0.952465
[12]	valid_0's auc: 0.952638
[13]	valid_0's auc: 0.95266
[14]	valid_0's auc: 0.952766
[15]	valid_0's auc: 0.953203
[16]	valid_0's auc: 0.953503
[17]	valid_0's auc: 0.953793
[18]	valid_0's auc: 0.953966
[19]	valid_0's auc: 0.954184
[20]	valid_0's auc: 0.9543
[21]	valid_0's auc: 0.954305
[22]	valid_0's auc: 0.954536
[23]	valid_0's auc: 0.954748
[24]	valid_0's auc: 0.955142
[25]	valid_0's auc: 0.955493
[26]	valid_0's auc: 0.955611
[27]	valid_0's auc: 0.955708
[28]	valid_0's auc: 0.955795
[29]	valid_0's auc: 0.956172
[30]	valid_0's auc: 0.95623
[31]	valid_0's auc: 0.956477
[32]	valid_0's auc: 0.956606
[33]	valid_0's auc: 0.9

In [11]:
from sklearn import metrics

ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['is_attributed'], ypred)
print(f"Test score: {score}")

Test score: 0.9717160293264153
